<a href="https://colab.research.google.com/github/rabinatwayana/DL_torchgeo_MMFlood_Segmentation/blob/master/Drive_mmflood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning-based Semantic Flood Mapping using UNet and SegFormer on Multimodal Earth Observation Data

Author: Rabina Twayana

This notebook aims to perform semantic segmentation for flood mapping by training, evaluating, and comparing different deep learning models.

## Install Packages

The notebook was run in Google Colab. Following packages were installed.

In [ ]:
!pip install torchgeo
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.3/859.3 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import tempfile
# tempfile.gettempdir()

'/tmp'

## Import Packages

In [ ]:
import torchgeo
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger ##https://lightning.ai/docs/pytorch/stable/extensions/generated/lightning.pytorch.loggers.WandbLogger.html#lightning.pytorch.loggers.WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torchgeo.trainers import SemanticSegmentationTask
from torch.utils.data import DataLoader
from datetime import datetime


# import wandb
# wandb.login()

# Project that the run is recorded to
# project = "MMFlood_DL_Experiments"


checkpoint_callback = ModelCheckpoint(
    monitor="val_iou",        # metric to monitor (IoU in your case)
    mode="max",               # save the checkpoint with max val_iou
    save_top_k=1,             # save only the best model
    filename="best-{epoch:02d}-{val_iou:.4f}"
)


# UNet run
# wandb_logger_unet = WandbLogger(
#     project="MMFlood_DL_Experiments",
#     name="unet"
# )

# SegFormer run
wandb_logger_segformer = WandbLogger(
    project="MMFlood_DL_Experiments",
    name="segformer"
)


trainer = Trainer(logger=wandb_logger_segformer, callbacks=[checkpoint_callback])

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


## Dataset
MMFlood dataset is a multimodal flood delineation dataset (Montello et al., 2022).

 Some Sentinel-1 tiles have missing data, which are automatically set to 0. Corresponding pixels in masks are set to 255 and should be ignored in performance computation.

Dataset features:

- 1,748 Sentinel-1 tiles of varying pixel dimensions

- Multimodal dataset (Sentinel-1, DEMs and hydrography maps (available for 1,012 tiles out of 1,748))

- 95 flood events from 42 different countries

- Flood delineation maps (ground truth) is obtained from Copernicus EMS

- Missing data in Sentinel-1 tiles are set to 0 and corrsponding pixels in masks are set to 255 (must ignored in performance computation)  

Dataset classes:

- no flood

- flood



In [ ]:
from torchgeo.datasets import MMFlood

In [ ]:
dataset = MMFlood(
    root="/content/sample_data/data",   # where data will be stored
    split="test",         # "train", "val", or "test"
    include_dem=True,      # optional
    include_hydro=False,   # optional
    download=True,         # 🔑 this triggers download
    checksum=True          # optional but recommended
)
MMFlood(download=True, include_dem=True, include_hydro=True)

100%|██████████| 1.07G/1.07G [00:13<00:00, 79.3MB/s]
100%|██████████| 1.07G/1.07G [00:14<00:00, 74.3MB/s]
100%|██████████| 1.07G/1.07G [00:09<00:00, 111MB/s]
100%|██████████| 1.07G/1.07G [00:23<00:00, 45.0MB/s]
100%|██████████| 1.07G/1.07G [00:40<00:00, 26.6MB/s]
100%|██████████| 1.07G/1.07G [00:44<00:00, 24.1MB/s]
100%|██████████| 1.07G/1.07G [00:41<00:00, 26.1MB/s]
100%|██████████| 1.07G/1.07G [00:52<00:00, 20.5MB/s]
100%|██████████| 1.07G/1.07G [00:45<00:00, 23.7MB/s]
100%|██████████| 1.07G/1.07G [00:46<00:00, 23.0MB/s]
100%|██████████| 423M/423M [00:19<00:00, 22.2MB/s]
100%|██████████| 63.0k/63.0k [00:00<00:00, 1.45MB/s]


Merging separate part files...
Processing file /content/sample_data/data/activations.tar.000.gz.part
Processing file /content/sample_data/data/activations.tar.001.gz.part
Processing file /content/sample_data/data/activations.tar.002.gz.part
Processing file /content/sample_data/data/activations.tar.003.gz.part
Processing file /content/sample_data/data/activations.tar.004.gz.part
Processing file /content/sample_data/data/activations.tar.005.gz.part
Processing file /content/sample_data/data/activations.tar.006.gz.part
Processing file /content/sample_data/data/activations.tar.007.gz.part
Processing file /content/sample_data/data/activations.tar.008.gz.part
Processing file /content/sample_data/data/activations.tar.009.gz.part
Processing file /content/sample_data/data/activations.tar.010.gz.part


100%|██████████| 1.07G/1.07G [00:11<00:00, 90.4MB/s]
100%|██████████| 1.07G/1.07G [00:14<00:00, 72.6MB/s]
100%|██████████| 1.07G/1.07G [00:16<00:00, 66.9MB/s]
100%|██████████| 1.07G/1.07G [00:11<00:00, 94.4MB/s]
100%|██████████| 1.07G/1.07G [00:09<00:00, 113MB/s]
100%|██████████| 1.07G/1.07G [00:10<00:00, 98.0MB/s]
100%|██████████| 1.07G/1.07G [00:21<00:00, 50.0MB/s]
100%|██████████| 1.07G/1.07G [00:13<00:00, 80.3MB/s]
100%|██████████| 1.07G/1.07G [00:11<00:00, 92.4MB/s]
100%|██████████| 1.07G/1.07G [00:10<00:00, 103MB/s]
100%|██████████| 423M/423M [00:04<00:00, 103MB/s]
100%|██████████| 63.0k/63.0k [00:00<00:00, 6.94MB/s]


Merging separate part files...
Processing file data/activations.tar.000.gz.part
Processing file data/activations.tar.001.gz.part
Processing file data/activations.tar.002.gz.part
Processing file data/activations.tar.003.gz.part
Processing file data/activations.tar.004.gz.part
Processing file data/activations.tar.005.gz.part
Processing file data/activations.tar.006.gz.part
Processing file data/activations.tar.007.gz.part
Processing file data/activations.tar.008.gz.part
Processing file data/activations.tar.009.gz.part
Processing file data/activations.tar.010.gz.part


OSError: [Errno 28] No space left on device

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Deeplearning Models

### a) Unet (Base model)

### b) SegFormer

## Train Model

In [ ]:


def train_model(model_name, input_type, train_dataset, val_dataset, max_epochs=50, batch_size=8):
    """
    Train a TorchGeo model (UNet, SegFormer) with dynamic wandb logging and checkpointing.

    Args:
        model_name (str): 'unet' or 'segformer'
        input_type (str): description of input bands, e.g., 's1_dem_hydro'
        train_dataset, val_dataset: PyTorch Datasets
        max_epochs (int): number of training epochs
        batch_size (int): batch size
    """

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Define dynamic wandb logger
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    wandb_logger = WandbLogger(
        project="MMFlood_DL_Experiments",
        name=f"{model_name}_{input_type}_{timestamp}"
    )

    # Define checkpoint callback
    checkpoint_callback = ModelCheckpoint(
        monitor="val_iou",
        mode="max",
        save_top_k=1,
        filename=f"best-{model_name}-{{epoch:02d}}-{{val_iou:.4f}}",
        dirpath=f"checkpoints/{model_name}_{input_type}"
    )

    # Define TorchGeo Trainer (LightningModule)
    task = SemanticSegmentationTask(
        model=model_name,
        in_channels=3,       # adjust based on your input
        num_classes=2,
        loss="ce",
        ignore_index=255
    )

    # Initialize PyTorch Lightning Trainer
    trainer = Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        logger=wandb_logger,
        callbacks=[checkpoint_callback]
    )

    # Fit model
    trainer.fit(task, train_loader, val_loader)

    # Return path of best checkpoint
    return checkpoint_callback.best_model_path


In [ ]:
# Example: train UNet
best_unet_ckpt = train_model(
    model_name="unet",
    input_type="s1_dem_hydro",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    max_epochs=50
)

# Example: train SegFormer
best_segformer_ckpt = train_model(
    model_name="segformer",
    input_type="s1_dem_hydro",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    max_epochs=50
)

print("Best UNet checkpoint:", best_unet_ckpt)
print("Best SegFormer checkpoint:", best_segformer_ckpt)


## References
Montello, F., Arnaudo, E., & Rossi, C. (2022). MMFlood: A Multimodal Dataset for Flood Delineation From Satellite Imagery. IEEE Access, 10, 96774–96787. https://doi.org/10.1109/ACCESS.2022.3205419

